In [1]:
import temporian as tp

import pandas as pd
import numpy as np

In [2]:
# Generate a synthetic dataset
timestamps = np.arange(0, 100, 0.1)
n = len(timestamps)
noise = 0.1 * np.random.randn(n)
trend = 0.01 * timestamps
season = 0.4 * np.sin(0.5 * timestamps)

raw_data = pd.DataFrame(
    {
        "timestamp": timestamps,
        # "noise": noise,
        # "trend": trend,
        # "seasoon": season,
        "signal": noise + trend + season,
    }
)

raw_data

,timestamp,signal
0,0.0,0.121797
1,0.1,-0.107223
2,0.2,-0.031710
3,0.3,0.092227
4,0.4,0.033460
...,...,...
995,99.5,0.867278
996,99.6,0.819647
997,99.7,0.863293
998,99.8,1.023265


In [3]:
# Convert the dataset into a Temporian dataset

source_data = tp.EventData.from_dataframe(raw_data)

source_data

data:
		index_names=[]
		feature_names=['signal']
    (): [ 0.   0.1  0.2 ... 99.7 99.8 99.9]
        signal<float64>: [ 0.1218 -0.1072 -0.0317 ...  0.8633  1.0233  0.7743])

In [4]:
# Plot the dataset
_ = source_data.plot()

In [5]:
# Compute some signals
source = source_data.schema()

signal = source["signal"]
trend = tp.simple_moving_average(signal, 50.0)
detrend = signal - trend
trend2 = tp.simple_moving_average(detrend, 10.0)
detrend2 = detrend - trend2
slope = trend2 - tp.sample(tp.lag(trend2, 5.0), trend2)

# TODO: Clean feature names with rename.
result = tp.glue(
    signal,
    tp.prefix("trend.", trend),
    tp.prefix("detrend.", detrend),
    tp.prefix("trend2.", trend2),
    tp.prefix("detrend2.", detrend2),
    tp.prefix("slope.", slope),
)

result_data = tp.evaluate(result, {source: source_data})

result_data

data:
		index_names=[]
		feature_names=['signal', 'trend.signal', 'detrend.sub_signal_signal', 'trend2.sub_signal_signal', 'detrend2.sub_sub_signal_signal_sub_signal_signal', 'slope.sub_sub_signal_signal_sub_signal_signal']
    (): [ 0.   0.1  0.2 ... 99.7 99.8 99.9]
        signal<float64>: [ 0.1218 -0.1072 -0.0317 ...  0.8633  1.0233  0.7743])
        trend.signal<float64>: [ 0.1218  0.0073 -0.0057 ...  0.746   0.7473  0.7482])
        detrend.sub_signal_signal<float64>: [ 0.     -0.1145 -0.026  ...  0.1173  0.276   0.0261])
        trend2.sub_signal_signal<float64>: [ 0.     -0.0573 -0.0468 ...  0.2352  0.233   0.228 ])
        detrend2.sub_sub_signal_signal_sub_signal_signal<float64>: [ 0.     -0.0573  0.0208 ... -0.1179  0.043  -0.2019])
        slope.sub_sub_signal_signal_sub_signal_signal<float64>: [    nan     nan     nan ... -0.0891 -0.0936 -0.0992])

In [6]:
# Plot the results
result_data.plot()